In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import sys
import numpy as np
from collections import OrderedDict as OD
#import uproot3
import uproot
import hist
import matplotlib.pyplot as plt
import mplhep as hep

#sAnaVersion = 'QCD_fullHT'



from HistogramListForPlotting_QCDStitch_HT import *

print(f"PlotHists1D_MC:: sAnaVersion: {sAnaVersion}")

if not os.path.exists(sOpDir):
    os.makedirs(sOpDir)
    
fIpFiles = OD()
for sIpFileName, sIpFileNameFull in sIpFiles.items():
    fIpFiles[sIpFileName] = uproot.open(sIpFileNameFull)

HistogramListForPlotting_QCDStitch_HT:: sAnaVersion: QCD_fullHT
HTRangesForPlotting = [[50, 150], [150, 250], [250, 350], [450, 550], [650, 750], [950, 1050], [1450, 1550], [1950, 2050]]
PlotHists1D_MC:: sAnaVersion: QCD_fullHT


In [3]:

era = 2018
luminosity = 59.83
cmsWorkStatus='Work in Progress'
sData = "" # "Data", "" 


#systematics_list = ['central']
marker_color_list = ['r', 'b', 'darkviolet', 'c', 'orange', 'green']
marker_style_list = ["o", "o", '>', '^', 'v', "s", "+", 'x', '*',"X"]
marker_size_list  = [3, 3, 2, 2, 2, 2, 2, 2, 2]
#marker_size_list  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
                
normalize_histogram = False
makeRatioPlot = True

printLevel = 5
skip_plotNameNice = [
    #"hLeadingPtGenBquark_pt",
    #"hLeadingPtGenBquark_eta",
    #"hLeadingPtGenBquarkHardSctred_pt",
    #"hLeadingPtGenBquarkHardSctred_eta",
    #"hLeadingPtGenBHadron_pt",
    #"hLeadingPtGenBHadron_eta",
    #"hLeadingPtGenBHadronStatus2_pt",
    #"hLeadingPtGenBHadronStatus2_eta",
]

for plotNameNice in histograms_dict.keys():
    if plotNameNice in skip_plotNameNice: continue
    
    if printLevel >= 0:
        print(f"plotNameNice: {plotNameNice}")
    
    xAxisLabel = histograms_dict[plotNameNice][sXLabel] if sXLabel in list(histograms_dict[plotNameNice].keys()) else None
    yAxisLabel = histograms_dict[plotNameNice][sYLabel] if sYLabel in list(histograms_dict[plotNameNice].keys()) else None
    xAxisRange = histograms_dict[plotNameNice][sXRange] if sXRange in list(histograms_dict[plotNameNice].keys()) else None
    yAxisRange = histograms_dict[plotNameNice][sYRange] if sYRange in list(histograms_dict[plotNameNice].keys()) else None    
        
    nRebins = histograms_dict[plotNameNice][sNRebin] if sNRebin in list(histograms_dict[plotNameNice].keys()) else 1
    
    #print(f"xAxisLabel: {xAxisLabel}, yAxisLabel: {yAxisLabel}, xAxisRange: {xAxisRange}, yAxisRange: {yAxisRange}, nRebins: {nRebins} ")
    
    histosToOverlay = OD()
    yAxisRange_auto = [1e20, -1e20]
    #for iHistoToOverlay_name in range(len(histograms_dict[plotNameNice][sHistosToOverlay])):
    for iHistoToOverlay_name in histograms_dict[plotNameNice][sHistosToOverlay].keys():
        #print(f"iHistoToOverlay_name: {iHistoToOverlay_name}")
        
        histosToOverlay[iHistoToOverlay_name] = None
        h_added = None
        h = []
        for iHistoToHadd in range(len(histograms_dict[plotNameNice][sHistosToOverlay][iHistoToOverlay_name])):
            
            sIpFileName_    = histograms_dict[plotNameNice][sHistosToOverlay][iHistoToOverlay_name][iHistoToHadd][sIpFileNameNice]
            fIp_            = fIpFiles[sIpFileName_]
            sHistogramName_ = histograms_dict[plotNameNice][sHistosToOverlay][iHistoToOverlay_name][iHistoToHadd][sHistName]
            h_ = fIp_[sHistogramName_]
            h1_ = h_.to_hist()
            
            h1Rebin_ = None
            if   nRebins == 1:
                h1Rebin_ = h1_
            elif   nRebins == 2:
                h1Rebin_ = h1_[::2j]
            elif nRebins == 3:
                h1Rebin_ = h1_[::3j]
            elif nRebins == 4:
                h1Rebin_ = h1_[::4j]
            elif nRebins == 5:
                h1Rebin_ = h1_[::5j]
            elif nRebins == 6:
                h1Rebin_ = h1_[::6j]
            elif nRebins == 10:
                h1Rebin_ = h1_[::10j]
            elif nRebins == 20:
                h1Rebin_ = h1_[::20j]
            elif nRebins == 40:
                h1Rebin_ = h1_[::40j]
            elif nRebins == 50:
                h1Rebin_ = h1_[::50j]
            elif nRebins == 100:
                h1Rebin_ = h1_[::100j]
            else:
                print(f"nRebins={nRebins} is not yet implemented... Implement it \t\t **** ERROR ****")
                break
                
            if   nRebins > 1:    
                h1_ = h1Rebin_
            
            if printLevel >= 0:
                print(f"sIpFileName_: {sIpFileName_},  sHistogramName_: {sHistogramName_}")
            '''
            print(f"h_ ({type(h_)}): {h_}")
            #print(f"h1_ ({type(h1_)}): {h1_}")
            print(f"h1_ ({type(h1_)})")
            print(f"h1_.values() ({type(h1_.values())}) ({len(h1_.values())}) {h1_.values()}")
            print(f"\nh1_.variances() ({type(h1_.variances())}) ({len(h1_.variances())}) {h1_.variances()}")
            
            print(f"h1_.view() ({type(h1_.view())}) ({len(h1_.view())}): {h1_.view()}")
            '''
            
            if printLevel >= 10:
                #print(f"h1_ ({type(h1_)}): {h1_}")
                print(f"h1_.values() ({type(h1_.values())}) ({len(h1_.values())}) {h1_.values()}")
                print(f"\nh1_.variances() ({type(h1_.variances())}) ({len(h1_.variances())}) {h1_.variances()}")
            
            
            h.append( h1_ )
            
            #if h_added == None:
            if histosToOverlay[iHistoToOverlay_name] == None:
                #h_added = h1_
                histosToOverlay[iHistoToOverlay_name] = h1_
            else:
                #h_added = h_added + h1_
                histosToOverlay[iHistoToOverlay_name] = histosToOverlay[iHistoToOverlay_name] + h1_
                
            #print(f"\n\nh_added.view() ({type(h_added.view())}) ({len(h_added.view())}): {h_added.view()}")
            
        if printLevel >= 8:
            print(f"histosToOverlay[{iHistoToOverlay_name}].values() ({type(histosToOverlay[iHistoToOverlay_name].values())}) ({len(histosToOverlay[iHistoToOverlay_name].values())}): {histosToOverlay[iHistoToOverlay_name].values()}")
            print(f"histosToOverlay[{iHistoToOverlay_name}].variances() ({type(histosToOverlay[iHistoToOverlay_name].variances())}) ({len(histosToOverlay[iHistoToOverlay_name].variances())}): {histosToOverlay[iHistoToOverlay_name].variances()}")
            
        idxXAxisRangeMin = 0
        idxXAxisRangeMax = -1
        if xAxisRange:
            # find yMin_ and yMax_ within xAxisRange
            xAxisBinCenters_ = histosToOverlay[iHistoToOverlay_name].axes[0].centers
            binWidth_        = abs(xAxisBinCenters_[0] - xAxisBinCenters_[1])/2
            idxXAxisRangeMin = np.argwhere(np.isclose(xAxisBinCenters_, xAxisRange[0], atol=binWidth_) ) # Returns: [[979]   [980]]
            idxXAxisRangeMax = np.argwhere(np.isclose(xAxisBinCenters_, xAxisRange[1], atol=binWidth_) ) # Returns: [[1019]  [1020]]
            idxXAxisRangeMin = idxXAxisRangeMin[0][0]
            idxXAxisRangeMax = idxXAxisRangeMax[-1][-1]
            
        yLow_ = histosToOverlay[iHistoToOverlay_name].values()[idxXAxisRangeMin:idxXAxisRangeMax+1] - np.sqrt(histosToOverlay[iHistoToOverlay_name].variances()[idxXAxisRangeMin:idxXAxisRangeMax+1])
        
        if yLow_[np.nonzero(yLow_)].shape[0] == 0: continue
                      
        yMin_ = np.amin( yLow_[np.nonzero(yLow_)] ) 
        yMax_ = np.amax( histosToOverlay[iHistoToOverlay_name].values()[idxXAxisRangeMin:idxXAxisRangeMax+1] + np.sqrt(histosToOverlay[iHistoToOverlay_name].variances()[idxXAxisRangeMin:idxXAxisRangeMax+1]) )
            
        yAxisRange_auto[0] = yMin_ if yMin_ < yAxisRange_auto[0] else yAxisRange_auto[0]
        yAxisRange_auto[1] = yMax_ if yMax_ > yAxisRange_auto[1] else yAxisRange_auto[1]
        
        if printLevel >= 5:
            print(f"{iHistoToOverlay_name = }: yAxisRange_auto: {yAxisRange_auto}, yMin_: {yMin_}, yMax_: {yMax_}, ")
        #print(f"histosToOverlay[iHistoToOverlay_name].values() - np.sqrt(histosToOverlay[iHistoToOverlay_name].variances() ({type(histosToOverlay[iHistoToOverlay_name].values() - np.sqrt(histosToOverlay[iHistoToOverlay_name].variances()))}): {histosToOverlay[iHistoToOverlay_name].values() - np.sqrt(histosToOverlay[iHistoToOverlay_name].variances())}")
        
        
        
      
    #hStack_list = [ hBkg_list[idx] for idx in idx_hBkg_sortedByIntegral ]  
    #sStack_list = [ sBkg_list[idx] for idx in idx_hBkg_sortedByIntegral ]  
    #print(f"histosToOverlay.values() ({type(histosToOverlay.values())}): histosToOverlay.values()")
    #print(f"list(histosToOverlay.values()) ({type(list(histosToOverlay.values()))}): list(histosToOverlay.values())")
    #print(f"histosToOverlay[0].values() ({type(histosToOverlay[0].values())}): {histosToOverlay[0].values()}")
    
    histosToOverlay_values_list    = np.array( [ histosToOverlay[iHistoToOverlay].values() for iHistoToOverlay in histosToOverlay.keys() ] )
    histosToOverlay_error_list     = np.array( [ np.sqrt(histosToOverlay[iHistoToOverlay].variances()) for iHistoToOverlay in histosToOverlay.keys() ] )
    histosToOverlay_binEdges       = histosToOverlay[list(histosToOverlay.keys())[0]].axes[0].edges
    histosToOverlay_name_list      = list(histosToOverlay.keys())
    #print(f"histosToOverlay_binEdges ({type(histosToOverlay_binEdges)}) ({len(histosToOverlay_binEdges)}): {histosToOverlay_binEdges}")
    
    if printLevel >= 3:
        print(f"yAxisRange: {yAxisRange},  yAxisRange_auto: {yAxisRange_auto}")

    for yAxisScale in ['linearY', 'logY']: #['linearY', 'logY']: # ['linearY']
        
        #fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(8,10), sharex='col', gridspec_kw={'height_ratios': [3, 1]}, subplot_kw={'ymargin': 0.4})
        fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(8,10), sharex='col', gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})

        for iHistoToOverlay in range(len(histosToOverlay_name_list)):
            hep.histplot(
                histosToOverlay_values_list[iHistoToOverlay], bins=histosToOverlay_binEdges, yerr=histosToOverlay_error_list[iHistoToOverlay], 
                label=histosToOverlay_name_list[iHistoToOverlay], 
                ax=ax[0], 
                histtype='errorbar', 
                density=normalize_histogram,
                color=marker_color_list[iHistoToOverlay], #'r',
                markersize=marker_size_list[iHistoToOverlay], #3,
                marker=marker_style_list[iHistoToOverlay] #marker_style_list
            )            
            
        
        if yAxisRange:
            yAxisRange_toUse = yAxisRange
        else:
            yAxisRange_toUse = yAxisRange_auto
            if yAxisScale == 'logY':
                yAxisRange_toUse[1] = yAxisRange_toUse[1] * 1e3
            else:
                yAxisRange_toUse[1] = yAxisRange_toUse[1] * 1.6
                
        if not normalize_histogram:
            #print(f"yAxisRange_toUse: {yAxisRange_toUse},  yAxisRange: {yAxisRange}, yAxisRange_auto: {yAxisRange_auto}")
            ax[0].set_ylim(yAxisRange_toUse[0], yAxisRange_toUse[1])
            #pass
        if xAxisRange: ax[0].set_xlim(xAxisRange[0], xAxisRange[1])
        if xAxisLabel: ax[0].set_xlabel(xAxisLabel)
        if xAxisLabel: ax[1].set_xlabel(xAxisLabel)
        if yAxisLabel: ax[0].set_ylabel(yAxisLabel)
        ax[1].set_ylabel("Ratio")
        ax[0].grid(True)
        ax[1].grid(True)
        
        
        # Ratio plot ---------------------------------------------------------------------
        if makeRatioPlot and len(histosToOverlay_name_list) > 1:
            yAxisRange_RatioPlot_auto = [1e10, -1e10]
            #for iHistoToOverlay in range(1, len(histosToOverlay_name_list)):
            for iHistoToOverlay in range(1, 2):
                N_values = histosToOverlay_values_list[iHistoToOverlay]
                D_values = histosToOverlay_values_list[0]
                N_errors = histosToOverlay_error_list[iHistoToOverlay]
                D_errors = histosToOverlay_error_list[0]
                
                ratio_values = np.divide(N_values, D_values, where=D_values!=0, out=np.ones(len(D_values)))
                ratio_error  = N_errors            
                ratio_error  = np.divide(ratio_error, D_values, where=D_values!=0, out=np.zeros(len(D_values)))

                #print(f"ratio_values ({type(ratio_values)}) ({len(ratio_values)}): {ratio_values}")
                #print(f"ratio_error ({type(ratio_error)}) ({len(ratio_error)}): {ratio_error}")
                    
                hep.histplot(
                    ratio_values, bins=histosToOverlay_binEdges, yerr=ratio_error, 
                    #label=histosToOverlay_name_list[iHistoToOverlay], 
                    ax=ax[1], 
                    histtype='errorbar', 
                    #density=normalize_histogram,
                    color=marker_color_list[iHistoToOverlay], #'r',
                    markersize=marker_size_list[iHistoToOverlay], #3,
                    marker=marker_style_list[iHistoToOverlay] #marker_style_list
                )
                
                yLow_ = ratio_values # ratio_values - ratio_error
                yUp_  = ratio_values # ratio_values + ratio_error
                yMin_ = np.amin( yLow_[np.nonzero(yLow_)] )
                yMax_ = np.amax( yUp_ )
            
                yAxisRange_RatioPlot_auto[0] = yMin_ if yMin_ < yAxisRange_RatioPlot_auto[0] else yAxisRange_RatioPlot_auto[0]
                yAxisRange_RatioPlot_auto[1] = yMax_ if yMax_ > yAxisRange_RatioPlot_auto[1] else yAxisRange_RatioPlot_auto[1]

    
            if xAxisRange: ax[1].set_xlim(xAxisRange[0], xAxisRange[1])
            ax[1].set_ylim(0, 2)
            ax[1].set_ylim(yAxisRange_RatioPlot_auto[0] * 0.98, yAxisRange_RatioPlot_auto[1] * 1.02)
            #ax[1].set_ylim(yAxisRange_RatioPlot_auto[0], yAxisRange_RatioPlot_auto[1])
            if xAxisLabel: ax[1].set_xlabel(xAxisLabel)
            #if yAxisLabel: ax[1].set_xlabel(xAxisLabel)
            ax[1].set_ylabel('Ratio')
                
            ax[1].axhline(y=1, linestyle='--')
            
        ax[0].legend(fontsize=15, loc='upper right', bbox_to_anchor=(0.4, 0.75, 0.6, 0.25), ncol=1)
        if yAxisScale == 'logY': ax[0].set_yscale('log', base=10)
        #ax[0].set_ymargin(1)

        #hep.cms.label(ax=ax[0], data=True if sData else False, year=era, lumi=luminosity, label=cmsWorkStatus, fontsize=14)
        ax[0].set_title(sAnaVersion)


        try:
            fig.savefig('%s/%s_%s.png' % (sOpDir, plotNameNice, yAxisScale), transparent=False, dpi=200, bbox_inches="tight")
        except:
            print("%s/%s_%s.png could not save" % (sOpDir, plotNameNice, yAxisScale))

        
        plt.close(fig)



plotNameNice: hGenLHE_HT50to150_all
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bEnrich/hGenLHE_HT_QCDStitchCutBQuarkPt_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bGen/hGenLHE_HT_QCDStitchCutBQuarkPt_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCDIncl/hGenLHE_HT_QCDStitchCutBQuarkPt_central
iHistoToOverlay_name = 'QCD Stitch bQuark pT (option 1)': yAxisRange_auto: [8695667900.336618, 657020493480.5292], yMin_: 8695667900.336618, yMax_: 657020493480.5292, 
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bEnrich/hGenLHE_HT_QCDStitchCutBHadron_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bGen/hGenLHE_HT_QCDStitchCutBHadron_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCDIncl/hGenLHE_HT_QCDStitchCutBHadron_central
iHistoToOverlay_name = 'QCD Stitch bHadron (option 2)': yAxisRange_auto: [8695667900.336618, 657020493480.5292], yMin_: 8710657795.997757, yMax_: 657020493480.5292, 
sIpFileName_: QCD_fullHT,  sHistogramName_: evt

plotNameNice: hGenLHE_HT250to350_QCDStitch
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bEnrich/hGenLHE_HT_QCDStitchCutBQuarkPt_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bGen/hGenLHE_HT_QCDStitchCutBQuarkPt_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCDIncl/hGenLHE_HT_QCDStitchCutBQuarkPt_central
iHistoToOverlay_name = 'QCD Stitch bQuark pT (option 1)': yAxisRange_auto: [136511710.9268292, 770260768.5690826], yMin_: 136511710.9268292, yMax_: 770260768.5690826, 
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bEnrich/hGenLHE_HT_QCDStitchCutBHadron_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bGen/hGenLHE_HT_QCDStitchCutBHadron_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCDIncl/hGenLHE_HT_QCDStitchCutBHadron_central
iHistoToOverlay_name = 'QCD Stitch bHadron (option 2)': yAxisRange_auto: [136138750.834702, 770260768.5690826], yMin_: 136138750.834702, yMax_: 769638631.8329467, 
yAxisRange: None,  yAxisRange_auto: [13613

plotNameNice: hGenLHE_HT650to750_QCD_bEnrich
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bEnrich/hGenLHE_HT_all_central
iHistoToOverlay_name = 'QCD_bEnrich all': yAxisRange_auto: [200918.297768097, 472223.59952714527], yMin_: 200918.297768097, yMax_: 472223.59952714527, 
yAxisRange: None,  yAxisRange_auto: [200918.297768097, 472223.59952714527]
plotNameNice: hGenLHE_HT650to750_QCD_bGen
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bGen/hGenLHE_HT_all_central
iHistoToOverlay_name = 'QCD_bGen all': yAxisRange_auto: [347044.0299985475, 777905.4228745198], yMin_: 347044.0299985475, yMax_: 777905.4228745198, 
yAxisRange: None,  yAxisRange_auto: [347044.0299985475, 777905.4228745198]
plotNameNice: hGenLHE_HT650to750_QCD_Incl
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCDIncl/hGenLHE_HT_all_central
iHistoToOverlay_name = 'QCDIncl all': yAxisRange_auto: [4324697.697483495, 9850524.33903857], yMin_: 4324697.697483495, yMax_: 9850524.33903857, 
yAxisRange: None,  yAxisRange_au

plotNameNice: hGenLHE_HT1450to1550_QCD_Incl
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCDIncl/hGenLHE_HT_all_central
iHistoToOverlay_name = 'QCDIncl all': yAxisRange_auto: [49884.28860319295, 77863.18988674313], yMin_: 49884.28860319295, yMax_: 77863.18988674313, 
yAxisRange: None,  yAxisRange_auto: [49884.28860319295, 77863.18988674313]
plotNameNice: hGenLHE_HT1950to2050_all
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bEnrich/hGenLHE_HT_QCDStitchCutBQuarkPt_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bGen/hGenLHE_HT_QCDStitchCutBQuarkPt_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCDIncl/hGenLHE_HT_QCDStitchCutBQuarkPt_central
iHistoToOverlay_name = 'QCD Stitch bQuark pT (option 1)': yAxisRange_auto: [3383.1194551783096, 5035.904026530738], yMin_: 3383.1194551783096, yMax_: 5035.904026530738, 
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_bEnrich/hGenLHE_HT_QCDStitchCutBHadron_central
sIpFileName_: QCD_fullHT,  sHistogramName_: evt/QCD_b